# atmodeller

## Tutorial 3: Monte Carlo experiment

We can devise a simple Monte Carlo approach to sample the probable atmospheres that can arise for different planetary conditions.

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [2]:
from atmodeller import InteriorAtmosphereSystem, Molecule, Planet, OCEAN_MOLES, SystemConstraint, logger
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2, BasaltLibourelN2, MolarMasses
import numpy as np
import csv
import logging

logger.setLevel(logging.INFO)

In [3]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='N2', solubility=BasaltLibourelN2(), solid_melt_distribution_coefficient=0))
molecules

[2023-06-04 14:08:50 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2O
[2023-06-04 14:08:50 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2
[2023-06-04 14:08:50 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: O2
[2023-06-04 14:08:50 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CO
[2023-06-04 14:08:50 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CO2
[2023-06-04 14:08:50 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: N2


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x1267302d0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(-3.0385132352941198, 13152.698529411768), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x111385a50>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x119d76d10>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x119da7190>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, for

In [4]:
planet: Planet = Planet()
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules, planet=planet)

[2023-06-04 14:08:50 - atmodeller.core      - 072 - INFO      - __post_init__()] Creating a new planet
[2023-06-04 14:08:50 - atmodeller.core      - 077 - INFO      - __post_init__()] Mantle mass (kg) = 4.208261222595111e+24
[2023-06-04 14:08:50 - atmodeller.core      - 078 - INFO      - __post_init__()] Mantle melt fraction = 1.0
[2023-06-04 14:08:50 - atmodeller.core      - 079 - INFO      - __post_init__()] Core mass fraction = 0.295334691460966
[2023-06-04 14:08:50 - atmodeller.core      - 080 - INFO      - __post_init__()] Planetary radius (m) = 6371000.0
[2023-06-04 14:08:50 - atmodeller.core      - 081 - INFO      - __post_init__()] Planetary mass (kg) = 5.972e+24
[2023-06-04 14:08:50 - atmodeller.core      - 082 - INFO      - __post_init__()] Surface temperature (K) = 2000.000000
[2023-06-04 14:08:50 - atmodeller.core      - 083 - INFO      - __post_init__()] Surface gravity (m/s^2) = 9.819973426224687
[2023-06-04 14:08:50 - atmodeller.core      - 084 - INFO      - __post_init_

In [5]:
number_of_realisations: int = 2

# Parameters are normally distributed between bounds.
number_ocean_moles: np.ndarray = np.random.uniform(1, 10, number_of_realisations)
ch_ratios: np.ndarray = np.random.uniform(0.1, 1, number_of_realisations)
fo2_shifts: np.ndarray = np.random.uniform(-4, 4, number_of_realisations)

# Store the output in a list.
out: list[dict[str, float]] = []

# ppmw of Nitrogen in the mantle. 2.8 is the mantle value of N.
N_ppmw: float = 2.8

# The nitrogen mass is constant.
mass_N: float = N_ppmw * 1.0e-6 * planet.mantle_mass

for realisation in range(number_of_realisations):

    mass_H: float = number_ocean_moles[realisation] * OCEAN_MOLES * MolarMasses().H2
    mass_C: float = ch_ratios[realisation] * mass_H
    constraints: list[SystemConstraint] = [
        SystemConstraint(species="H", value=mass_H, field="mass"),
        SystemConstraint(species="C", value=mass_C, field="mass"),
        SystemConstraint(species="N", value=mass_N, field="mass")
    ]
    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    planet.fo2_shift = fo2_shifts[realisation]
    out_realisation = interior_atmosphere.solve(constraints, fo2_constraint=True)

    # Include the parameters in the output.
    out_realisation['number_ocean_moles'] = number_ocean_moles[realisation]
    out_realisation['ch_ratio'] = ch_ratios[realisation]
    out_realisation['fo2_shift'] = planet.fo2_shift

    out.append(out_realisation)

    filename: str = "atmodeller_monte_carlo_tutorial3.csv"
    print("Writing output to: %s", filename)
    fieldnames: list[str] = list(out[0].keys())
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer: csv.DictWriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(out)

[2023-06-04 14:08:50 - atmodeller.core      - 756 - INFO      - solve()] Constraints: [SystemConstraint(species='H', value=1.2363228293180007e+21, field='mass'),
 SystemConstraint(species='C', value=1.1795132190705819e+21, field='mass'),
 SystemConstraint(species='N', value=1.178313142326631e+19, field='mass')]
[2023-06-04 14:08:50 - atmodeller.core      - 782 - INFO      - solve()] Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
[2023-06-04 14:08:50 - atmodeller.core      - 557 - INFO      - get_coefficient_matrix_and_rhs()] Adding fO2 as an additional constraint using IronWustiteBufferOneill with fO2_shift = 3.25
[2023-06-04 14:08:50 - atmodeller.core      - 582 - INFO      - get_coefficient_matrix_and_rhs()] 3 additional (not pressure) constraint(s) are necessary to solve the system
[2023-06-04 14:08:50 - atmodeller.core      - 591 - INFO      - get_coefficient_matrix_and_rhs()] Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
[2023-06-04 14:

Writing output to: %s equilibrium_atmosphere_monte_carlo.csv
Writing output to: %s equilibrium_atmosphere_monte_carlo.csv


The simulation data is output as:

In [6]:
out

[{'CO': 81.91890192428191,
  'H2': 0.8351893960539017,
  'N2': 3.0832966409731357,
  'O2': 6.8949164417799e-05,
  'CO2': 522.8458026601214,
  'H2O': 23.926848779582702,
  'number_ocean_moles': 7.976592868485964,
  'ch_ratio': 0.9540495339079381,
  'fo2_shift': 3.2463528495404335},
 {'CO': 47.39408715730857,
  'H2': 0.3811609464135545,
  'N2': 2.843142889824771,
  'O2': 5.169156926031599e-06,
  'CO2': 82.82453784828911,
  'H2O': 2.989883623345757,
  'number_ocean_moles': 2.811291257573422,
  'ch_ratio': 0.5966919279986818,
  'fo2_shift': 2.121243558875393}]